# Hyperparameter tuning

## Part 1: Manual Hyperparameter Tuning

### Objective

Manually tune hyperparameters of a neural network and observe the impact on model performance.

### Setup

Start with the necessary imports and dataset preparation. We'll use the MNIST dataset for this exercise, as it's complex enough to demonstrate the effects of hyperparameter tuning.


In [64]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train.reshape(-1, 784) / 255.0, X_test.reshape(-1, 784) / 255.0
# y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

## Task: Manual Tuning of Hyperparameters

1. Build a Base Model: Create a simple neural network as a starting point.
2. Manual Tuning: Experiment by manually changing hyperparameters like learning rate, number of layers/neurons, and activation functions.
3. Training and Evaluation: Train the model with different hyperparameter settings and evaluate its performance.


In [55]:
# def build_model(num_layers, learning_rate):
#     # Construct a model based on hyperparams
#     model = Sequential([
#         Dense(num_layers[0], activation='relu', input_shape=(784,)),
#         Dense(num_layers[1], activation='relu'),
#         Dense(10, activation='softmax')
#     ])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])
#     return model

# # Example hyperparameters to tune
# learning_rates = [0.001, 0.01]
# layer_configs = [(32, 32), (64, 64)]

# # Loop through different hyperparameters and train models
# for lr in learning_rates:
#     for layers in layer_configs:
#         # Build and train your model
#         build_model(layers, lr)

In [56]:
# def train_and_evaluate(model, epochs=50):
#     history = model.fit(X_train, y_train, epochs=epochs, validation_split=0.2)
#     test_loss, test_acc = model.evaluate(X_test, y_test)
#     return history, test_loss, test_acc

In [57]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(16, 10))

# # Loop through different hyperparameters
# i = 1  # Index for subplot
# for lr in learning_rates:
#     for layers in layer_configs:
#         plt.subplot(3, 3, i)  # 3x3 grid of subplots

#         model = build_model(layers, lr)
#         history, loss, accuracy = train_and_evaluate(model)

#         plt.plot(history.history['val_accuracy'])
#         plt.plot(history.history['accuracy'])
#         plt.title("LR: {}, Layers: {}, Val Acc: {:.2f}".format(lr, layers, history.history['val_accuracy'][-1]))
#         plt.xlabel('Epochs')
#         plt.ylabel('Val Accuracy')

#         i += 1  # Increment subplot index

# plt.tight_layout()
# plt.show()

## Visualization

Plot the accuracy and loss for different hyperparameter settings.

## Analysis and Questions

- How did different learning rates affect the training process and model accuracy?
- What impact did varying the number of layers and neurons have on the model's performance?
- Were there any combinations of hyperparameters that resulted in particularly good or poor performance?


---

## Part 2: Automated Hyperparameter Tuning

### Objective

Use automated methods like Grid Search and Random Search for hyperparameter tuning.

### Setup

Reuse the MNIST dataset setup from Part 2.


### Task: Automated Hyperparameter Tuning

1. Grid Search and Random Search: Introduce and apply Grid Search and Random Search using scikit-learn's GridSearchCV or RandomizedSearchCV.
2. Integration with Keras: Show how to use these methods with Keras models.


In [58]:
# print(X_train.shape, y_train.shape)

In [59]:
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# from scikeras.wrappers import KerasClassifier


# # Define a function to create a model (for KerasClassifier)
# def create_model_to_search(num_layers, learning_rate, activation, dropout_rate, optimizer, loss, metrics, l1, l2, **kwargs):
#     # Convert optimizer string to actual optimizer object
#     if optimizer == 'adam':
#         optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#     elif optimizer == 'sgd':
#         optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

#     # Build the model
#     model = Sequential([
#         Dense(num_layers[0], activation=activation, input_shape=(784,)),
#         tf.keras.layers.Dropout(dropout_rate),
#         Dense(num_layers[1], activation=activation,
#               kernel_regularizer=tf.keras.regularizers.L1(l1),
#               bias_regularizer=tf.keras.regularizers.L2(l2),
#               activity_regularizer=tf.keras.regularizers.L1L2(l1, l2)),
#         tf.keras.layers.Dropout(dropout_rate),
#     ])
#     model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
#     return model


# # Set up GridSearchCV or RandomizedSearchCV
# model_to_search = KerasClassifier(build_fn=create_model_to_search)
# param_grid = {
#     'num_layers': [(32, 32), (64, 64), (128, 128)],
#     'learning_rate': [0.001, 0.01],
#     'activation': ['relu', 'tanh'],
#     'dropout_rate': [0.2, 0.5],
#     'optimizer': ['adam', 'sgd'],
#     'loss': ['categorical_crossentropy'],
#     'metrics': [['accuracy']],
#     'l1': [0.01, 0.001],
#     'l2': [0.01, 0.001]
# }


# grid = GridSearchCV(estimator=model_to_search, param_grid=param_grid)

# # Run grid search
# grid_result = grid.fit(X_train, y_train)

In [65]:
import keras_tuner as kt


def build_model_hp(hp):
    model = Sequential()
    # model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    model.add(Dense(28, activation="relu", input_shape=(784,)))

    hp_units = hp.Int("units", min_value=32, max_value=512, step=32)
    model.add(Dense(units=hp_units, activation="relu"))
    model.add(Dense(units=hp_units, activation="relu"))
    model.add(Dense(10, activation="softmax"))

    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    return model

In [61]:
tuner = kt.Hyperband(
    build_model_hp,
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="my_dir",
    project_name="hyper_param_tuning",
)

Reloading Tuner from my_dir/hyper_param_tuning/tuner0.json


In [62]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

In [66]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 26 Complete [00h 00m 30s]
val_accuracy: 0.9472500085830688

Best val_accuracy So Far: 0.9746666550636292
Total elapsed time: 05h 00m 10s


In [67]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

{'units': 512, 'learning_rate': 0.001, 'tuner/epochs': 10, 'tuner/initial_epoch': 4, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0019'}


In [68]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history["val_accuracy"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print("Best epoch: %d" % (best_epoch,))

Epoch 1/50


/Users/Cyrille/.local/share/virtualenvs/Deep_learning-ZjjgEFKy/lib/python3.11/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1500/1500 [==============================] - 6s 3ms/step - loss: 0.2850 - accuracy: 0.9128 - val_loss: 0.1603 - val_accuracy: 0.9496
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1433 - accuracy: 0.9561 - val_loss: 0.1414 - val_accuracy: 0.9568
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1076 - accuracy: 0.9662 - val_loss: 0.1264 - val_accuracy: 0.9617
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0867 - accuracy: 0.9716 - val_loss: 0.1276 - val_accuracy: 0.9638
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0727 - accuracy: 0.9771 - val_loss: 0.1093 - val_accuracy: 0.9694
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0625 - accuracy: 0.9796 - val_loss: 0.1141 - val_accuracy: 0.9670
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0560 - accuracy: 0.9819 - val_loss: 0.1112 - val_accuracy: 0.9709
Epo

In [69]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/29


/Users/Cyrille/.local/share/virtualenvs/Deep_learning-ZjjgEFKy/lib/python3.11/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1500/1500 [==============================] - 6s 3ms/step - loss: 0.2897 - accuracy: 0.9104 - val_loss: 0.1672 - val_accuracy: 0.9479
Epoch 2/29
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1423 - accuracy: 0.9553 - val_loss: 0.1428 - val_accuracy: 0.9562
Epoch 3/29
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1100 - accuracy: 0.9649 - val_loss: 0.1203 - val_accuracy: 0.9643
Epoch 4/29
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0868 - accuracy: 0.9729 - val_loss: 0.1057 - val_accuracy: 0.9692
Epoch 5/29
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0746 - accuracy: 0.9762 - val_loss: 0.1222 - val_accuracy: 0.9646
Epoch 6/29
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0654 - accuracy: 0.9787 - val_loss: 0.1093 - val_accuracy: 0.9716
Epoch 7/29
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0560 - accuracy: 0.9814 - val_loss: 0.0989 - val_accuracy: 0.9714
Epo

In [70]:
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

 34/313 [==>...........................] - ETA: 0s - loss: 0.1901 - accuracy: 0.9724 

/Users/Cyrille/.local/share/virtualenvs/Deep_learning-ZjjgEFKy/lib/python3.11/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 1ms/step - loss: 0.1738 - accuracy: 0.9726
[test loss, test accuracy]: [0.17383113503456116, 0.972599983215332]


testing with gridsearch


In [86]:
# Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train.reshape(-1, 784) / 255.0, X_test.reshape(-1, 784) / 255.0
# y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

In [87]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scikeras.wrappers import KerasClassifier


def create_model_to_search(
    num_layers=32,
    learning_rate=0.001,
    activation="relu",
    dropout_rate=0.2,
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics="accuracy",
    l1=0.01,
    l2=0.01,
):
    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model = Sequential(
        [
            Dense(num_layers, activation=activation, input_shape=(784,)),
            tf.keras.layers.Dropout(dropout_rate),
            Dense(
                num_layers,
                activation=activation,
                kernel_regularizer=tf.keras.regularizers.L1(l1),
                bias_regularizer=tf.keras.regularizers.L2(l2),
                activity_regularizer=tf.keras.regularizers.L1L2(l1, l2),
            ),
            tf.keras.layers.Dropout(dropout_rate),
        ]
    )
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model


model_to_search = KerasClassifier(
    build_fn=create_model_to_search,
    verbose=0,
    num_layers=32,
    learning_rate=0.001,
    activation="relu",
    dropout_rate=0.2,
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics="accuracy",
    l1=0.01,
    l2=0.01,
)
param_grid = {
    "num_layers": [32, 64, 128],
    "learning_rate": [0.001, 0.01],
    "activation": ["relu", "tanh"],
    "dropout_rate": [0.2, 0.5],
    "optimizer": ["adam", "sgd"],
    "loss": ["categorical_crossentropy"],
    "metrics": ["accuracy"],
    "l1": [0.01, 0.001],
    "l2": [0.01, 0.001],
}


grid = GridSearchCV(estimator=model_to_search, param_grid=param_grid)

# Run grid search
grid_result = grid.fit(X_train, y_train)

/Users/Cyrille/.local/share/virtualenvs/Deep_learning-ZjjgEFKy/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/Cyrille/.local/share/virtualenvs/Deep_learning-ZjjgEFKy/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/Cyrille/.local/share/virtualenvs/Deep_learning-ZjjgEFKy/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/Cyrille/.local/share/virtualenvs/Deep_learning-ZjjgEFKy/lib

ValueError: 
All the 960 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
320 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/Cyrille/.local/share/virtualenvs/Deep_learning-ZjjgEFKy/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/Cyrille/.local/share/virtualenvs/Deep_learning-ZjjgEFKy/lib/python3.11/site-packages/scikeras/wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/Cyrille/.local/share/virtualenvs/Deep_learning-ZjjgEFKy/lib/python3.11/site-packages/scikeras/wrappers.py", line 760, in fit
    self._fit(
  File "/Users/Cyrille/.local/share/virtualenvs/Deep_learning-Zjj

## Visualization

Visualize the performance of the best model found by the search methods.

## Analysis and Questions

- Compare the results of manual tuning with automated tuning. Which method gave better results?
- What are the advantages and limitations of using automated methods like Grid Search and Random Search?
